In [ ]:
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from urllib import parse
from collections import Counter

In [ ]:
log_lines = []
with open("data/logs_all_searches.txt") as file:
    for line in file:
        log_lines.append(line)



In [ ]:
log_lines[0].split("\t")

In [ ]:
re.search("GET /judgments/(advanced_)?search\\?([^\\s]+) HTTP/1\.1", log_lines[0])[2]

In [ ]:
re.search("GET /judgments/(advanced_)?search\\?([^\\s]+) HTTP/1\.1", log_lines[0])[2]

In [ ]:
def get_query(line):
    match = re.search("GET /judgments/advanced_search\\?([^\\s]+) HTTP/1\.1", line)
    if match and match[1]:
        return dict(parse.parse_qsl(match[1]))
        


In [ ]:
queries = []
for line in log_lines:
    queries.append(get_query(line))
queries

In [ ]:
total = 0
per_param = Counter()
for query in queries:
    if query is not None:
        total += 1
        for key in query:
            per_param[key] += 1

In [ ]:
per_param

In [ ]:
total

In [ ]:
per_param['specific_keyword'] / float(total)

In [ ]:
per_param["neutral_citation"] / float(total)

In [ ]:
params_counts_df = pd.DataFrame.from_dict(per_param, orient='index', columns=["n"]).reindex()

In [ ]:
params_counts_df["percentage"] = params_counts_df["n"] / total * 100

In [ ]:
params_counts_df.sort_values("n", ascending=False)

In [ ]:
log_lines[0]

In [ ]:
log_lines[-1]

In [ ]:
total

In [ ]:
params_df = pd.DataFrame.from_records([query for query in queries if query is not None])

In [ ]:
params_df

In [ ]:
params_df.groupby("specific_keyword").count().rename({"page": "n"}, axis=1)[["n"]].sort_values("n", ascending=False)

In [ ]:
as_first_page_counts = pd.DataFrame(params_df[pd.isna(params_df["page"])].count(), columns=["n"])

In [ ]:
as_first_page_counts["percentage"] = as_first_page_counts["n"] / float(params_df[pd.isna(params_df["page"])].shape[0]) * 100

In [ ]:
float(params_df[pd.isna(params_df["page"])].shape[0])

In [ ]:
as_first_page_counts.sort_values("n", ascending=False)

In [ ]:
as_first_page = pd.DataFrame(params_df[pd.isna(params_df["page"]) & pd.isna(params_df["per_page"]) & pd.isna(params_df["order"])])
as_first_page["i"] = 1

In [ ]:
as_first_page.groupby("specific_keyword").count().rename({"court": "n"}, axis=1)[["n"]].sort_values("n", ascending=False)

In [ ]:
as_first_page.groupby("judge").count().rename({"i": "n"}, axis=1)[["n"]].sort_values("n", ascending=False)

In [ ]:
as_first_page_no_fuzz = as_first_page[~as_first_page.apply(lambda r: r.str.contains('FUZZ', case=False).any(), axis=1)] 

In [ ]:
as_first_page_no_fuzz_count = pd.DataFrame(as_first_page_no_fuzz.count(), columns=["n"])

In [ ]:
as_first_page_no_fuzz_total = as_first_page_no_fuzz.shape[0]; as_first_page_no_fuzz_total

In [ ]:
as_first_page_no_fuzz_count["percentage"] = as_first_page_no_fuzz_count["n"] / as_first_page_no_fuzz_total * 100

In [ ]:
as_first_page_no_fuzz_count.sort_values("n", ascending=False)

In [ ]:
params_df["n"] = 1

In [ ]:
pages = params_df[~params_df["page"].str.contains("FUZZ", na=False)].fillna("0").groupby("page").count()[["n"]].reset_index()
pages["page"] = pages["page"].astype(int)
pages.sort_values("page", ascending=True).set_index("page").plot(logy=True)

In [ ]:
pages[pages["page"] > 200]

In [ ]:
params_df_unfuzzed = params_df[~params_df.apply(lambda r: r.str.contains('FUZZ', case=False).any(), axis=1)] 

In [ ]:
params_df_unfuzzed["real_page"] = params_df_unfuzzed["page"].fillna(0).astype(int)

In [ ]:
params_df_unfuzzed.group_by("real_page").count()[["n"]]

In [ ]:
params_df_unfuzzed[params_df_unfuzzed["real_page"] > 200].shape[0] / params_df_unfuzzed.shape[0]

In [ ]:
bot_param_counts = pd.DataFrame(params_df_unfuzzed[params_df_unfuzzed["real_page"] > 200].count(), columns=["n"])

In [ ]:
params_df_unfuzzed.groupby("query").count()[["n"]].sort_values("n", ascending=False).head(20)

In [ ]:
queries

In [ ]:
queryterms = set()
for query in queries:
    if query:
        if "query" in query:
            queryterms.add(query["query"])

In [ ]:
for term in queryterms:
    print(term)

In [ ]:
log_lines

In [ ]:
"a moron in a hurry", 3, "https:/balaslkajlaskjflsdjfs///"